In [2]:
import sys
sys.path.insert(0,'/home/jindal/notebooks/jindal/NER/')
import os
import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from keras.optimizers import Adam
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
import subprocess

/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
epochs = 80
trainable = True

In [4]:
from keras_contrib.layers import CRF


In [5]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [6]:
# changing all deriv and part to misc. with BIO
def modify_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset
                

In [7]:

# preproecessing data from GermEval
def get_sentences(path):
    sentences=[]
    with open(path,'rb') as f:
    #     lines = f.readlines()
        sentence=[]
        for line in f:
            try:
                splits = [x.decode() for x in line.split()]
            except Exception as e:
                continue
            try:
                if splits[0]!='#':
                    temp = [splits[1],splits[2]]
                    sentence.append(temp)
                else:
                    if len(sentence):
                        sentences.append(sentence)
                    sentence=[]
            except Exception as e:
                continue
    return sentences



In [112]:

# preproecessing data from Conll
def get_sentences(filename):
    '''
        -DOCSTART- -X- -X- O

    EU NNP B-NP B-ORG
    rejects VBZ B-VP O
    German JJ B-NP B-MISC
    call NN I-NP O
    to TO B-VP O
    boycott VB I-VP O
    British JJ B-NP B-MISC
    lamb NN I-NP O
    . . O O
    
    '''
    
    '''
    read file
    return format :
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]
    '''
    f = open(filename,'rb')
    sentences = []
    sentence = []
    for line in f:
        splits = line.split()
        try:
            word=splits[0].decode()
            if word=='-DOCSTART-':
                continue
            label=splits[-1].decode()
            temp=[word,label]
            sentence.append(temp)
        except Exception as e:
            if len(sentence)!=0:
                sentences.append(sentence)
                sentence=[]
    return sentences



In [8]:
train_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-train.tsv')
dev_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-dev.tsv')
test_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-test.tsv')

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


23999
2199
5099


In [113]:
# conll
train_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.train')
dev_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.testa')
test_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.testb')

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


12152
2867
3005


In [9]:
print(train_sentences[100])

[['Der', 'O'], ['Meteoritensucher', 'O'], ['hatte', 'O'], ['im', 'O'], ['Frühjahr', 'O'], ['auf', 'O'], ['der', 'O'], ['dänischen', 'B-LOCderiv'], ['Insel', 'O'], ['Lolland', 'B-LOC'], ['30', 'O'], ['Gramm', 'O'], ['eines', 'O'], ['Meteoriten', 'O'], ['aufgespürt', 'O'], [',', 'O'], ['der', 'O'], ['Mitte', 'O'], ['Januar', 'O'], ['über', 'O'], ['Nordeuropa', 'B-LOC'], ['beobachtet', 'O'], ['worden', 'O'], ['war', 'O'], ['.', 'O']]


In [115]:
# train_sentences = modify_labels(train_sentences)
# dev_sentences = modify_labels(dev_sentences)
# test_sentences = modify_labels(test_sentences)

In [10]:
trainSentences = addCharInformatioin(train_sentences)
devSentences = addCharInformatioin(dev_sentences)
testSentences = addCharInformatioin(test_sentences)


In [11]:
labelSet = set()
words = {}
characters= set()

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token, char, label in sentence:
            for c in char:
                characters.add(c)
            labelSet.add(label)
            words[token.lower()] = True
            # words[token] = True

In [12]:
print(labelSet)

{'B-OTHpart', 'O', 'B-ORGpart', 'B-PERpart', 'B-LOCderiv', 'I-ORGderiv', 'I-OTH', 'B-OTHderiv', 'B-PERderiv', 'I-OTHderiv', 'I-ORGpart', 'B-OTH', 'B-ORGderiv', 'I-LOC', 'I-PERpart', 'I-LOCpart', 'I-PERderiv', 'B-PER', 'B-LOCpart', 'I-LOCderiv', 'B-ORG', 'B-LOC', 'I-ORG', 'I-PER', 'I-OTHpart'}


In [13]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [14]:
print(label2Idx)

{'B-OTHpart': 0, 'B-ORGpart': 2, 'I-OTHderiv': 9, 'B-LOCderiv': 4, 'I-ORGderiv': 5, 'I-ORGpart': 10, 'B-OTH': 11, 'I-OTH': 6, 'B-ORGderiv': 12, 'O': 1, 'I-LOC': 13, 'I-PERpart': 14, 'I-PERderiv': 16, 'B-PER': 17, 'B-LOCpart': 18, 'B-OTHderiv': 7, 'I-LOCderiv': 19, 'B-PERpart': 3, 'B-ORG': 20, 'B-LOC': 21, 'I-ORG': 22, 'B-PERderiv': 8, 'I-PER': 23, 'I-LOCpart': 15, 'I-OTHpart': 24}


In [15]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [18]:
string_words = ' '.join(words.keys())
# print(string_words)
f = open('/home/jindal/notebooks/fastText/german_words.txt','wb')
f.write(string_words.encode())
f.close()

In [16]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'numeric': 0, 'PADDING_TOKEN': 7, 'contains_digit': 6, 'mainly_numeric': 5, 'initialUpper': 3, 'allUpper': 2, 'other': 4, 'allLower': 1}


In [19]:
word2Idx={}
wordEmbeddings=[]

# created a file by the name of german_words.txt in /fastText. Containing all the words in our dataset
with open('/home/jindal/notebooks/jindal/NER/ablation_study/german_word_embeddings.txt','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
#         print(word.decode())
        if len(word2Idx) == 0: #Add padding+unknown
            word2Idx["PADDING_TOKEN"] = len(word2Idx)
            vector = np.zeros(len(splits)-1) #Zero vector vor 'PADDING' word
            wordEmbeddings.append(vector)

            word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
            vector = np.random.uniform(-0.25, 0.25, len(splits)-1)
            wordEmbeddings.append(vector)
            
        word2Idx[word.lower()]=len(word2Idx)
        embedding = np.array([float(num) for num in splits[1:]])
        wordEmbeddings.append(embedding)
wordEmbeddings=np.array(wordEmbeddings)

In [ ]:
word2Idx={}
wordEmbeddings=[]

fEmbeddings = open("/home/jindal/notebooks/embeddings/embeddings/model_word2vecf.embeddings", encoding="utf-8")

for line in fEmbeddings:
    split = line.strip().split(" ")
    word = split[0].lower()
#     print(word)
    
    if len(word2Idx) == 0: #Add padding+unknown
        word2Idx["PADDING_TOKEN"] = len(word2Idx)
        vector = np.zeros(len(split)-1) #Zero vector vor 'PADDING' word
        wordEmbeddings.append(vector)
        
        word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
        vector = np.random.uniform(-0.25, 0.25, len(split)-1)
        wordEmbeddings.append(vector)

    if split[0].lower() in words:
        vector = np.array([float(num) for num in split[1:]])
        wordEmbeddings.append(vector)
        word2Idx[word] = len(word2Idx)
wordEmbeddings=np.array(wordEmbeddings)

In [20]:
# print(wordEmbeddings[2])
print(len(wordEmbeddings))
print(len(wordEmbeddings[2]))

86688
300


In [21]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
print(char2Idx)

{'대': 0, 'Î': 1, 'Ö': 110, '守': 3, '½': 285, 'ö': 4, 'м': 6, '⊃': 7, 'ň': 192, '佐': 8, 'ŏ': 321, '¹': 9, 'æ': 19, 'ą': 12, 'α': 15, '0': 14, '=': 16, 'オ': 17, 'Ġ': 18, '妃': 290, 'Λ': 20, 'A': 21, 'ş': 22, '博': 270, '€': 28, '학': 30, 'q': 26, '-': 27, 'ʻ': 29, 'И': 176, '!': 24, 'В': 31, 'ô': 32, '\x95': 228, '4': 33, '£': 34, 'ę': 35, '"': 36, '術': 230, 'ś': 174, 'Ш': 39, '’': 40, 'û': 49, 'b': 43, "'": 45, '算': 52, 'j': 46, 'ї': 322, 'Z': 48, 'Č': 50, 'ب': 54, 'ő': 233, 'Â': 57, 'я': 274, 'σ': 55, '−': 56, 'M': 58, 'x': 60, '太': 231, 'Á': 65, '→': 66, 'з': 67, 'Æ': 68, '.': 69, 'è': 276, 'É': 71, 'ь': 73, '貴': 119, 'υ': 296, 'Ł': 239, 'k': 74, 'ā': 238, '\x80': 76, 'ъ': 226, 'ñ': 77, '(': 78, '‚': 79, 'Y': 80, 'Π': 314, '章': 81, '별': 83, '台': 240, 'O': 84, 'o': 86, 'в': 88, 'õ': 89, '&': 10, 'J': 90, 'Ä': 122, 'ħ': 92, '±': 94, 'κ': 97, '樓': 98, 'ě': 262, 'F': 324, 'š': 99, 'п': 75, 'ë': 101, '李': 103, 'ε': 282, 'u': 104, '\xad': 105, 'ラ': 106, 'б': 38, 'Š': 109, 'ж': 272, '懿': 277, '

In [22]:
print(trainSentences[0]) 

[['Schartau', ['S', 'c', 'h', 'a', 'r', 't', 'a', 'u'], 'B-PER'], ['sagte', ['s', 'a', 'g', 't', 'e'], 'O'], ['dem', ['d', 'e', 'm'], 'O'], ['"', ['"'], 'O'], ['Tagesspiegel', ['T', 'a', 'g', 'e', 's', 's', 'p', 'i', 'e', 'g', 'e', 'l'], 'B-ORG'], ['"', ['"'], 'O'], ['vom', ['v', 'o', 'm'], 'O'], ['Freitag', ['F', 'r', 'e', 'i', 't', 'a', 'g'], 'O'], [',', [','], 'O'], ['Fischer', ['F', 'i', 's', 'c', 'h', 'e', 'r'], 'B-PER'], ['sei', ['s', 'e', 'i'], 'O'], ['"', ['"'], 'O'], ['in', ['i', 'n'], 'O'], ['einer', ['e', 'i', 'n', 'e', 'r'], 'O'], ['Weise', ['W', 'e', 'i', 's', 'e'], 'O'], ['aufgetreten', ['a', 'u', 'f', 'g', 'e', 't', 'r', 'e', 't', 'e', 'n'], 'O'], [',', [','], 'O'], ['die', ['d', 'i', 'e'], 'O'], ['alles', ['a', 'l', 'l', 'e', 's'], 'O'], ['andere', ['a', 'n', 'd', 'e', 'r', 'e'], 'O'], ['als', ['a', 'l', 's'], 'O'], ['überzeugend', ['ü', 'b', 'e', 'r', 'z', 'e', 'u', 'g', 'e', 'n', 'd'], 'O'], ['war', ['w', 'a', 'r'], 'O'], ['"', ['"'], 'O'], ['.', ['.'], 'O']]


In [23]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = padding(createMatrices(trainSentences, word2Idx,  label2Idx, case2Idx, char2Idx))
dev_set = padding(createMatrices(devSentences, word2Idx, label2Idx, case2Idx, char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx, label2Idx, case2Idx, char2Idx))

In [27]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
print((train_set[0][2]))
print(len(train_set[2][3]))

[[44340, 42288, 77401, 2402, 61580, 2402, 15841, 27569, 46353, 22589, 1811, 2402, 4259, 22653, 21388, 37191, 46353, 64772, 72220, 62513, 11802, 75317, 41711, 2402, 70303], [3, 1, 1, 4, 3, 4, 1, 3, 4, 3, 1, 4, 1, 1, 3, 1, 4, 1, 1, 1, 1, 1, 1, 4, 4], array([[211, 308, 316, ...,   0,   0,   0],
       [  5, 252, 209, ...,   0,   0,   0],
       [299, 113, 164, ...,   0,   0,   0],
       ...,
       [281, 252, 182, ...,   0,   0,   0],
       [ 36,   0,   0, ...,   0,   0,   0],
       [ 69,   0,   0, ...,   0,   0,   0]], dtype=int32), [17, 1, 1, 1, 20, 1, 1, 1, 1, 17, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
25
[[211 308 316 ...   0   0   0]
 [  5 252 209 ...   0   0   0]
 [299 113 164 ...   0   0   0]
 ...
 [281 252 182 ...   0   0   0]
 [ 36   0   0 ...   0   0   0]
 [ 69   0   0 ...   0   0   0]]
24


In [28]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [29]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [30]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
# dropout= Dropout(0.5, name='dropout1')(embed_char_out)

# conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
# maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
# char = TimeDistributed(Flatten())(maxpool_out)
# char = Dropout(0.5)(char)
# output = concatenate([words, casing, char])

kernel_sizes = (3, 4, 5)
conv_blocks = []
for sz in kernel_sizes:
    conv = TimeDistributed(Conv1D(
                         kernel_size=sz,
                         filters=32,
                         padding="same",
                         activation="relu",
                         strides=1))(embed_char_out)
    conv = TimeDistributed(MaxPooling1D(52))(conv)
    conv = TimeDistributed(Flatten())(conv)
#     conv = Dropout(0.5)(conv)
    conv_blocks.append(conv)
output = concatenate([words, casing, conv_blocks[0], conv_blocks[1], conv_blocks[2]])

output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5))(output)
output = TimeDistributed(Dense(len(label2Idx)))(output)
crf = CRF(len(label2Idx))
output = crf(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
# model.compile(loss=crf.loss_function, optimizer=Adam(lr = 0.002), metrics=[crf.accuracy])
model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
model.summary()
# plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10496       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_4 (TimeDistrib (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distr

In [35]:
for x in range(15,20):
#     print(i)
    maxf1 = 0
    for epoch in range(epochs):    
        print("Epoch %d/%d"%(epoch, epochs))
        a = Progbar(len(train_batch_len))
        for i, batch in enumerate(iterate_minibatches(train_batch, train_batch_len)):
            labels, tokens, casing, char = batch
            labels = labels.tolist()
    #         print(labels)
            for sentence in labels:
                for i,label in enumerate(sentence):
                    temp = [0]*len(label2Idx)
                    value = label[0]
                    temp[value]=1
                    sentence[i] = temp
            labels = np.array(labels)
    #         print(labels)
            model.train_on_batch([tokens, casing, char], labels)
            a.update(i)
        if epoch >= 50:
            predLabels, correctLabels = tag_dataset(dev_batch)        
            pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
#             predLabels = np.concatenate(predLabels).ravel()
#             correctLabels = np.concatenate(correctLabels).ravel()
            print(len(predLabels), len(correctLabels))
#             f1_dev = sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' )
            print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))
            print(f1_dev)

            if f1_dev > maxf1:
                model.save('german_ner_conll.h5')
                maxf1 = f1_dev
    #         predLabels, correctLabels = tag_dataset(dev_batch)        
        #     with open("dropout=0.75.txt",'w') as f:
    #         pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
        #     with open('dropout=0.75.txt') as f:
        #         x = str(epoch)+ " "+f1_dev
        #         f.write(x)
        #         f.write('\n')
    #         print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))


    id2word = {v: k for k, v in word2Idx.items()}
    print(len(correctLabels))        

    model.load_weights('german_ner_conll.h5')
    predLabels, correctLabels = tag_dataset(test_batch)        
    # pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
    # print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

    id = 1
    file_name='test_conll'+str(x)+'.txt'
    input_file_name = '/home/jindal/notebooks/jindal/NER/'+file_name
    f = open(input_file_name,'wb')
    for sentence_number, batch in enumerate(test_batch):
        for word_number, wordid in enumerate(batch[0]):
            word = id2word[wordid]
            true_label = correctLabels[sentence_number][word_number]
            true_label = idx2Label[true_label]
            pred_label = predLabels[sentence_number][word_number]
            pred_label = idx2Label[pred_label]

#             string = str(id) + '\t' + word + '\t' + true_label +'\t' + true_label+'\t'+pred_label+'\t'+pred_label+'\t\n'
            string = word + ' ' + pred_label+' '+pred_label+' '+true_label+'\n'
            string = string.encode()
            f.write(string)
            id+=1

    # command = "perl nereval.perl"
    # directory_germeval = r'/home/jindal/notebooks/Resources/GermEVAL'
    directory_conll = r'/home/jindal/notebooks/Resources/CONLL/2003/ner/bin'
    command = "perl conlleval"
    
    output_file_name = '/home/jindal/notebooks/jindal/NER/test_result_conll'+str(x)+'.txt'
    output_file=open(output_file_name,'w')
    input_file = open(input_file_name)
    import subprocess
    process = subprocess.Popen(command.split(), stdin=input_file, stdout=output_file, cwd=directory_conll)
    out, err = process.communicate()
#     print(out)
    print('******************************************************************************')

Epoch 0/80
55/51 [================================] - 12s 226ms/step
Epoch 1/80
55/51 [================================] - 13s 236ms/step
Epoch 2/80
55/51 [================================] - 12s 221ms/step
Epoch 3/80
55/51 [================================] - 13s 234ms/step
Epoch 4/80
55/51 [================================] - 13s 229ms/step
Epoch 5/80
55/51 [================================] - 13s 229ms/step
Epoch 6/80
55/51 [================================] - 12s 224ms/step
Epoch 7/80
55/51 [================================] - 13s 231ms/step
Epoch 8/80
55/51 [================================] - 13s 239ms/step
Epoch 9/80
55/51 [================================] - 12s 226ms/step
Epoch 10/80
55/51 [================================] - 13s 232ms/step
Epoch 11/80
55/51 [================================] - 13s 239ms/step
Epoch 12/80
55/51 [================================] - 13s 238ms/step
Epoch 13/80
55/51 [================================] - 13s 236ms/step
Epoch 14/80
55/51 [===========

KeyboardInterrupt: 

In [76]:
import os
os.getcwd()

'/home/jindal/notebooks/Resources/GermEVAL'